In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# 1. Tải dữ liệu từ file CSV
# Thay 'path/to/your/data.csv' bằng đường dẫn file thực tế của bạn trên Kaggle
df = pd.read_csv('/kaggle/input/data-do-an-cs221/batch_processed_data.csv')

# Kiểm tra dữ liệu (Optional)
print("Số lượng mẫu:", len(df))
df.head()

# 2. Chuyển đổi sang định dạng Dataset của Hugging Face
dataset = Dataset.from_pandas(df)

# 3. Chia tập train/validation (ví dụ lấy 10% để validate)
dataset = dataset.train_test_split(test_size=0.1)

2025-12-19 08:18:57.050522: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766132337.224365      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766132337.275535      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Số lượng mẫu: 1000


In [2]:
# Tải tokenizer của T5-base
model_checkpoint = "VietAI/vit5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Định nghĩa token ngăn cách. 
# Bài báo dùng ký hiệu [s], ta có thể dùng token đặc biệt hoặc chuỗi string.
# Ở đây ta dùng chuỗi " <sep> " để mô hình dễ học.
SEPARATOR = " <sep> "

def preprocess_function(examples):
    inputs = []
    targets = []
    
    for i in range(len(examples['input'])):
        src = str(examples['input'][i])
        cand3 = str(examples['candidate_3'][i])
        target = str(examples['output'][i])

        # CHỈ dùng source + candidate_3
        combined_input = f"{src}{SEPARATOR}{cand3}"

        inputs.append(combined_input)
        targets.append(target)
    # Tokenize input và output
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    
    # Tokenize targets (output mong muốn)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=1024, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Áp dụng hàm xử lý lên toàn bộ dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [3]:
# Tải mô hình T5-base
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Định nghĩa các tham số huấn luyện (Hyperparameters)
# Bạn có thể điều chỉnh batch_size và learning_rate tùy theo GPU trên Kaggle (P100/T4)
args = Seq2SeqTrainingArguments(
    output_dir="lmcor-t5-finetuned",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,          # Learning rate phổ biến cho T5
    per_device_train_batch_size=4, # Giảm xuống 4 nếu gặp lỗi OOM (hết bộ nhớ)
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,          # Chỉ lưu 3 checkpoint gần nhất để tiết kiệm ổ cứng
    num_train_epochs=3,          # Số vòng lặp huấn luyện
    predict_with_generate=True,  # Để tính metrics khi eval
    fp16=True,                   # Sử dụng mixed precision để chạy nhanh hơn trên GPU
    report_to="none"             # Tắt log lên wandb nếu không cần thiết
)

# Data Collator giúp đệm (padding) dữ liệu cho đều nhau trong mỗi batch
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Khởi tạo Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/904M [00:00<?, ?B/s]

/tmp/ipykernel_20/2210717053.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [4]:
# Bắt đầu training
trainer.train()

# Lưu model sau khi train xong
trainer.save_model("final_lmcor_model")
print("Đã huấn luyện và lưu model thành công!")

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,0.301782
2,No log,0.290663
3,0.421500,0.299377


Đã huấn luyện và lưu model thành công!


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

# 1. Đường dẫn đến model đã lưu (đảm bảo trùng với tham số bạn dùng trong trainer.save_model)
MODEL_PATH = "final_lmcor_model"  # Hoặc thư mục checkpoint gần nhất
SEPARATOR = " <sep> " # QUAN TRỌNG: Phải giống hệt lúc train

# 2. Load Tokenizer và Model
print(f"Đang tải model từ {MODEL_PATH}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)

# 3. Khởi tạo Pipeline
# device=0 nghĩa là dùng GPU đầu tiên, device=-1 là dùng CPU
device = 0 if torch.cuda.is_available() else -1
corrector = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=device
)
print(f"✓ Đã load model thành công trên device={'GPU' if device==0 else 'CPU'}")

Đang tải model từ final_lmcor_model...


Device set to use cuda:0


✓ Đã load model thành công trên device=GPU


In [6]:
!pip install errant

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mkl-fft to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-random to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-umath to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.3/499.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 109.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that ar

In [7]:
import errant
import spacy
import sys
import subprocess

# --- HÀM HỖ TRỢ CÀI ĐẶT VÀ CHẤM ĐIỂM ---
def ensure_spacy_model(model_name='en_core_web_sm'):
    try:
        spacy.load(model_name)
    except OSError:
        print(f"⚠ Đang cài đặt {model_name}...")
        subprocess.check_call([sys.executable, "-m", "spacy", "download", model_name])

def evaluate_metrics(df, eval_cols):
    """
    Tính điểm Precision, Recall, F0.5 cho danh sách các cột trong DataFrame
    """
    ensure_spacy_model()
    print("⏳ Đang load ERRANT (có thể mất vài giây)...")
    annotator = errant.load('en')

    results = {}

    for col in eval_cols:
        print(f"▶ Đang chấm điểm cột: {col}")
        tp, fp, fn = 0, 0, 0

        # Dùng tqdm để hiện thanh tiến trình
        for _, row in tqdm(df.iterrows(), total=len(df), leave=False):
            src = str(row['input'])
            ref = str(row['output'])
            hyp = str(row[col])

            if pd.isna(hyp): hyp = ""

            # Parse
            orig = annotator.parse(src)
            cor = annotator.parse(hyp)
            gold = annotator.parse(ref)

            # Annotate edits
            hyp_edits = annotator.annotate(orig, cor)
            gold_edits = annotator.annotate(orig, gold)

            # Convert to set for comparison
            hyp_set = set([(e.o_start, e.o_end, e.c_str) for e in hyp_edits])
            gold_set = set([(e.o_start, e.o_end, e.c_str) for e in gold_edits])

            tp += len(hyp_set & gold_set)
            fp += len(hyp_set - gold_set)
            fn += len(gold_set - hyp_set)

        # Calculate metrics
        p = tp / (tp + fp) if (tp + fp) > 0 else 0
        r = tp / (tp + fn) if (tp + fn) > 0 else 0
        beta = 0.5
        f05 = (1 + beta**2) * (p * r) / ((beta**2 * p) + r) if (p + r) > 0 else 0

        results[col] = {'P': p, 'R': r, 'F0.5': f05}
        print(f"   Done {col}: F0.5 = {f05:.4f}")

    return results

In [8]:
from tqdm.auto import tqdm
import pandas as pd

test_dict={"llama":"/kaggle/input/data-do-an-cs221/batch_processed_data_fpt_llama.csv",
          "gemini-2.5":"/kaggle/input/data-do-an-cs221/batch_processed_data_test.csv",
          "gemini-2.0":"/kaggle/input/data-do-an-cs221/batch_processed_data_test_gemini20flash.csv",
           "gemma":"/kaggle/input/cs221dataaddiaitonla/batch_processed_data_fpt_gemma.csv",
           "qwen":"/kaggle/input/cs221datagwen/batch_processed_data_fpt_qwen.csv"
          }
import os

# Danh sách các cột cần so sánh sau khi inference
cols_to_compare = ['candidate_1', 'candidate_2', 'candidate_3', 'lmcor_output']
all_summaries = []

# Duyệt qua từng bộ test set trong từ điển đã định nghĩa
for test_name, file_path in test_dict.items():
    print(f"\n{'='*20} Đang xử lý tập: {test_name} {'='*20}")
    
    # 1. Tải dữ liệu
    try:
        current_df = pd.read_csv(file_path)
    except Exception as e:
        print(f"❌ Không thể tải file {file_path}: {e}")
        continue
        
    print(f"Số lượng mẫu: {len(current_df)}")
    lmcor_outputs = []

    # 2. Chạy Inference (Loop qua từng dòng)
    for index, row in tqdm(current_df.iterrows(), total=len(current_df), desc=f"Inference {test_name}"):
        try:
            src = str(row['input'])
            cand3 = str(row['candidate_3'])

            # CHỈ dùng source + candidate_3, giống lúc train
            prompt = f"{src}{SEPARATOR}{cand3}"
            
            # Dự đoán với max_new_tokens=128 [cite: 27]
            result = corrector(prompt, max_new_tokens=128)
            pred_text = result[0]['generated_text'] if isinstance(result, list) else result['generated_text'] [cite: 28]
        except Exception as e:
            pred_text = src  # Lỗi thì giữ nguyên câu gốc
            
        lmcor_outputs.append(pred_text)

    # 3. Lưu kết quả Inference của tập này 
    current_df['lmcor_output'] = lmcor_outputs
    output_filename = f"results_{test_name}.csv"
    current_df.to_csv(output_filename, index=False)
    print(f"✅ Đã lưu kết quả tại: {output_filename}")

    # 4. Đánh giá Metrics (P, R, F0.5) [cite: 20, 21, 31]
    metrics_results = evaluate_metrics(current_df, cols_to_compare)
    
    # Tạo bảng báo cáo cho tập hiện tại [cite: 31, 32]
    print(f"\n--- BÁO CÁO KẾT QUẢ: {test_name} ---")
    print(f"{'MODEL / CANDIDATE':<20} | {'PRECISION':<10} | {'RECALL':<10} | {'F0.5 SCORE':<10}")
    
    for name, m in metrics_results.items():
        print(f"{name:<20} | {m['P']:.4f}     | {m['R']:.4f}     | {m['F0.5']:.4f}")
        # Lưu lại để tổng hợp sau này
        all_summaries.append({
            'Test Set': test_name,
            'Model': name,
            'Precision': m['P'],
            'Recall': m['R'],
            'F0.5': m['F0.5']
        })

# 5. Lưu bảng tổng hợp báo cáo của tất cả các tập
summary_df = pd.DataFrame(all_summaries)
summary_df.to_csv("all_test_sets_summary.csv", index=False)
print("\n" + "!"*30)
print("Tất cả các tập dữ liệu đã được xử lý và lưu báo cáo tổng hợp!")


==================== Đang xử lý tập: llama ====================
Số lượng mẫu: 500


Inference llama:   0%|          | 0/500 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ Đã lưu kết quả tại: results_llama.csv
⏳ Đang load ERRANT (có thể mất vài giây)...
▶ Đang chấm điểm cột: candidate_1


  0%|          | 0/500 [00:00<?, ?it/s]

...

   Done candidate_2: F0.5 = 0.0004
▶ Đang chấm điểm cột: candidate_3


  0%|          | 0/500 [00:00<?, ?it/s]

   Done candidate_3: F0.5 = 0.0010
▶ Đang chấm điểm cột: lmcor_output


  0%|          | 0/500 [00:00<?, ?it/s]

   Done lmcor_output: F0.5 = 0.1102

--- BÁO CÁO KẾT QUẢ: qwen ---
MODEL / CANDIDATE    | PRECISION  | RECALL     | F0.5 SCORE
candidate_1          | 0.0007     | 0.0011     | 0.0008
candidate_2          | 0.0003     | 0.0006     | 0.0004
candidate_3          | 0.0009     | 0.0014     | 0.0010
lmcor_output         | 0.1033     | 0.1510     | 0.1102

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Tất cả các tập dữ liệu đã được xử lý và lưu báo cáo tổng hợp!
